In [1]:
!pip install torch
import gym
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import time

from collections import deque

You should consider upgrading via the 'C:\Users\jayam\anaconda3\python.exe -m pip install --upgrade pip' command.


C:\Users\jayam\anaconda3\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\jayam\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
GAMMA = 0.99
LEARNING_RATE = 0.01
TARGET_UPDATE = 10

n_episodes = 100000
hidden_dim = 12
min_eps = 0.01
max_eps_episode = 50

In [3]:
import torch.nn as nn

class QNN(nn.Module):
    def __init__(self, ip_dim, op_dim, hid_dim) -> None:
        
        super(QNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(ip_dim, hid_dim),
            torch.nn.PReLU()
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(hid_dim, hid_dim),
            torch.nn.PReLU()
        )

        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(hid_dim, hid_dim),
            torch.nn.PReLU()
        )
        
        self.final = torch.nn.Linear(hid_dim, op_dim)
        
    def forward(self, t: torch.Tensor) -> torch.Tensor:
        t = self.layer1(t)
        t = self.layer2(t)
        t = self.layer3(t)
        t = self.final(t)

        return t
        

        

In [4]:
import random
from collections import namedtuple

transition = namedtuple('transition',('state', 'action', 'reward','nxt_state','stop'))

class ReplayMemory_CyclicBuffer(object):
    
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity) 
            
    def __len__(self):
        return len(self.memory)
        
    def push_to_memory(self, *args):
        self.memory.append(transition(*args))
        
    def random_sampling(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    
# test = transition(((1,2),(2,3),(4,5)),('stay','left','right'),(1,1,1),((4,5),(6,7),(8,9)))
# print(test)
    

In [5]:
import torch.optim as optim

from torch.autograd import Variable

BATCH_SIZE = 64
FloatTensor = torch.FloatTensor #32-bit float multi dimension matrix

class Agent_MountainCart(object):
    
    def __init__(self, n_states, n_actions, hidden_dim, learning_rate):
        self.device = torch.device("cpu")
        self.local_q = QNN(n_states, n_actions, hidden_dim).to(self.device)
        self.target_q = QNN(n_states, n_actions, hidden_dim).to(self.device)
        
        self.mse_loss = torch.nn.MSELoss()
        self.optim = optim.Adam(self.local_q.parameters(), lr=learning_rate)
        
        self.n_states = n_states
        self.n_actions = n_actions
        
        self.replay_memory = ReplayMemory_CyclicBuffer(10000)
        
        
    def get_action(self, state, eps_greedy, eps_flag = True):
        global steps_done
        
        random_sample = random.random()
        
        if random_sample > eps_greedy:
            with torch.no_grad():
                return self.local_q((state).type(FloatTensor)).data.max(1)[1].view(1,1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]])
        
    def learn(self, experience, GAMMA):
        
        if len(self.replay_memory.memory) < BATCH_SIZE:
            return;
        
        transitions_sample = self.replay_memory.random_sampling(BATCH_SIZE)
        batch = transition(*zip(*transitions_sample))
        
        states = torch.cat(batch.state)
        actions = torch.cat(batch.action)
        rewards = torch.cat(batch.reward)
        nxt_states = torch.cat(batch.nxt_state)
        stops = torch.cat(batch.stop)
        
        expected_Q = torch.gather(self.local_q(states),1,actions)
        nxt_targets_Q = self.target_q(nxt_states).detach().max(1)[0]
        targets_Q = rewards + (GAMMA * nxt_targets_Q * (1-stops)) #1-dones
        self.local_q.train(mode = True)
        self.optim.zero_grad()
        loss = self.mse_loss(expected_Q, targets_Q.unsqueeze(1))
        loss.backward()
        self.optim.step()
        

In [6]:


env = gym.make('MountainCar-v0')

dim_space = env.observation_space.shape[0]
dim_action = env.action_space.n

threshold = env.spec.reward_threshold

print('ip_dim:', dim_space, 'op_dim:', dim_action,'hidden_dim:',hidden_dim,'threshold:', threshold)

agent = Agent_MountainCart(dim_space, dim_action, hidden_dim, learning_rate=LEARNING_RATE)

def epsilon_annealing(i_episode, max_episode, min_eps: float):
    
    slope = (min_eps - 1.0)/max_episode
    max_eps = max(slope * i_episode + 1.0, min_eps)
    return max_eps

def save_q(directory, filename):
    torch.save(agent.local_q.state_dict(), '%s/%s_local.pth' % (directory, filename))
    torch.save(agent.target_q.state_dict(), '%s/%s_target.pth' % (directory, filename))

    
    

ip_dim: 2 op_dim: 3 hidden_dim: 12 threshold: -110.0


C:\Users\jayam\anaconda3\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\jayam\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [7]:
def play_episode(env, agent, eps):
    
    state = env.reset()
    stop = False
    total_reward = 0
    
    while not stop:
        action = agent.get_action(FloatTensor([state]), eps)
        nxt_state, reward, stop, _ = env.step(action.item())
        total_reward = total_reward + reward
        
        if stop:
            reward = -1
            
        agent.replay_memory.push_to_memory(
            FloatTensor([state]),
             action,
             FloatTensor([reward]),
             FloatTensor([nxt_state]),
             FloatTensor([stop])
        )
        
        if(len(agent.replay_memory) > BATCH_SIZE):
            batch = agent.replay_memory.random_sampling(BATCH_SIZE)
            agent.learn(batch, GAMMA)
            
        state = nxt_state
        
    return total_reward

In [8]:
def train():
    scores = deque(maxlen=100)
    temp_scores = []
    avg_scores = []
    
    start_time = time.time()
    
    for i_episode in range(n_episodes):
        eps = epsilon_annealing(i_episode, max_eps_episode, min_eps)
        score = play_episode(env, agent, eps)
        
        scores.append(score)
        temp_scores.append(score)
        
        avg_score = np.mean(scores)
        avg_scores.append(avg_score)
        
        dt = (int)(time.time() - start_time)
        
        if i_episode % 10 == 0 and i_episode > 0:
            print('Episode: {:5} Score: {:5}  Avg.Score: {:.2f}, eps-greedy: {:5.3f} Time: {:02}:{:02}:{:02}'.\
                    format(i_episode, score, avg_score, eps, dt//3600, dt%3600//60, dt%60))
            
        if len(scores) == 100 and  avg_score >= threshold: 
                print('\n MountainCart solved in {:d} episodes!\tAverage Score: {:.2f}'. \
                    format(i_episode, np.mean(scores)))
                break
                
        if i_episode % TARGET_UPDATE == 0:
            agent.target_q.load_state_dict(agent.local_q.state_dict()) 
                
    return temp_scores, avg_scores

scores, avg_scores = train()
            

C:\Users\jayam\AppData\Local\Temp/ipykernel_1356/437941733.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  action = agent.get_action(FloatTensor([state]), eps)


Episode:    10 Score: -200.0  Avg.Score: -200.00, eps-greedy: 0.802 Time: 00:00:04
Episode:    20 Score: -200.0  Avg.Score: -200.00, eps-greedy: 0.604 Time: 00:00:09
Episode:    30 Score: -200.0  Avg.Score: -200.00, eps-greedy: 0.406 Time: 00:00:13
Episode:    40 Score: -200.0  Avg.Score: -200.00, eps-greedy: 0.208 Time: 00:00:18
Episode:    50 Score: -200.0  Avg.Score: -198.39, eps-greedy: 0.010 Time: 00:00:23
Episode:    60 Score: -200.0  Avg.Score: -198.66, eps-greedy: 0.010 Time: 00:00:29
Episode:    70 Score: -200.0  Avg.Score: -197.37, eps-greedy: 0.010 Time: 00:00:35
Episode:    80 Score: -200.0  Avg.Score: -195.04, eps-greedy: 0.010 Time: 00:00:40
Episode:    90 Score: -117.0  Avg.Score: -194.67, eps-greedy: 0.010 Time: 00:00:46
Episode:   100 Score: -200.0  Avg.Score: -193.08, eps-greedy: 0.010 Time: 00:00:54
Episode:   110 Score: -200.0  Avg.Score: -193.08, eps-greedy: 0.010 Time: 00:01:01
Episode:   120 Score: -200.0  Avg.Score: -193.08, eps-greedy: 0.010 Time: 00:01:07
Epis

In [9]:
torch.save(agent.target_q,"MountainCarModel")

In [10]:
agent.target_q = torch.load("MountainCarModel")


In [11]:
score = play_episode(env, agent, 0.01)

In [12]:
score

-79.0

In [ ]:
print('scores length: ', len(scores), ', avg_scores length: ', len(avg_scores))
plt.plot(np.arange(1, len(avg_scores)+1), avg_scores, label="Avg on 100 episodes")
plt.ylabel('Score')
plt.xlabel('Episodes #')
plt.title('DQN Convergence (Avg Score: -109.99 | Episodes: 1539) with 16 hidden dimensions\n')
plt.show()

scores length:  785 , avg_scores length:  785
